In [10]:
genres = ['pop', 'metal', 'rock', 'country', 'rap']

## Processing metal
https://www.kaggle.com/datasets/sebastianeck/refined-large-metal-lyrics-archive-228k-songs

In [103]:
import pandas as pd

metal_data = pd.read_csv('./data/metal_lyrics.csv')

# metal_data = metal_data[metal_data['DetectedLanguage'] == 'en']


# metal_data.drop(columns=[ 'Artist','Album','Song','SongNum','Year', 'Certainty','DetectedLanguage'], inplace = True)
# metal_data['genre'] = 'Metal'
# metal_data.to_csv('./metal_lyrics.csv', index = False)

## Processing Country, Pop, Rock, Rap
https://huggingface.co/datasets/amishshah/song_lyrics

In [104]:
from datasets import load_dataset

dataset = load_dataset("amishshah/song_lyrics")

In [105]:
dataset = dataset.filter(lambda song: song['tag'] in genres)

In [106]:
import tqdm.auto as tqdm
import pandas as pd

genre_counts = {'rap': [], 'pop': [], 'rock': [], 'country': []}

# Iterate over the selected samples and add them to the DataFrame

def get_genre_samples(sample):
    global genre_counts
    tag = sample['tag']
    if len(genre_counts[tag]) < 120000 or (tag == 'rap' and len(genre_counts[tag]) < 250000):
        genre_counts[tag].append([sample['lyrics'], sample['tag']])

dataset.map(get_genre_samples)

Map:   0%|          | 0/3078130 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'title', 'tag', 'lyrics'],
        num_rows: 3078130
    })
})

In [107]:
lyrics_df = pd.read_csv('./data/metal_lyrics.csv', index_col = False)

In [108]:
temp_df = pd.DataFrame(genre_counts['rap'], columns=lyrics_df.columns)
lyrics_df = pd.concat([lyrics_df, temp_df], ignore_index=True)

temp_df = pd.DataFrame(genre_counts['country'], columns=lyrics_df.columns)
lyrics_df = pd.concat([lyrics_df, temp_df], ignore_index=True)

temp_df = pd.DataFrame(genre_counts['pop'], columns=lyrics_df.columns)
lyrics_df = pd.concat([lyrics_df, temp_df], ignore_index=True)

temp_df = pd.DataFrame(genre_counts['rock'], columns=lyrics_df.columns)
lyrics_df = pd.concat([lyrics_df, temp_df], ignore_index=True)


lyrics_df.to_csv('./data/lyrics.csv', index = False)

# More country
https://huggingface.co/datasets/chloeliu/lyrics/viewer/default/train?f%5Bgenre%5D%5Bvalue%5D=%27country%27&row=7058

In [109]:
dataset = load_dataset("chloeliu/lyrics")

In [110]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'artist_name', 'track_name', 'release_date', 'genre', 'lyrics', 'len', 'dating', 'violence', 'world/life', 'night/time', 'shake the audience', 'family/gospel', 'romantic', 'communication', 'obscene', 'music', 'movement/places', 'light/visual perceptions', 'family/spiritual', 'like/girls', 'sadness', 'feelings', 'danceability', 'loudness', 'acousticness', 'instrumentalness', 'valence', 'energy', 'topic', 'age'],
        num_rows: 28372
    })
})

In [111]:
country_df = pd.DataFrame({ 'Lyric': dataset['train']['lyrics'], 'genre': dataset['train']['genre'] })
country_df = country_df[country_df['genre'].isin(genres)]

In [113]:
lyrics_df = pd.read_csv('./data/lyrics.csv', index_col = False)
lyrics_df.groupby('genre').size()

genre
Metal      179655
country     86658
pop        120000
rap        250000
rock       120000
dtype: int64

In [114]:
lyrics_df = pd.concat([lyrics_df, country_df], ignore_index=True)

In [115]:
from datasets import load_dataset

dataset_new = load_dataset("ThatOneShortGuy/SongLyrics")

In [116]:
country_df = pd.DataFrame({ 'Lyric': dataset_new['train']['lyrics'], 'genre': dataset_new['train']['genre'] })
country_df = country_df[country_df['genre'] == 'Country']
country_df['genre'] = 'country'

In [117]:
lyrics_df = pd.concat([lyrics_df, country_df], ignore_index=True)
lyrics_df.drop_duplicates(inplace = True)

In [118]:
lyrics_df.groupby('genre').size()

genre
Metal      176626
country    134607
pop        124731
rap        249494
rock       123844
dtype: int64

In [119]:
# lyrics_df = pd.read_csv('./lyrics.csv', index_col = False)
# Define the words to exclude
words_to_exclude = ['nigger', 'fagot', 'pussy', 'dick', 'nigga', 'slut', 'whore']

# Filter rows where the 'genre' column does not contain any of the words to exclude
lyrics_df = lyrics_df[~lyrics_df['Lyric'].str.contains('|'.join(words_to_exclude))]

lyrics_df

,Lyric,genre
0,"Etsin nutta pelkoa, syvyytta Mutta en ainoasta...",Metal
1,Drops of new gleaming In the flesh new morning...,Metal
2,Electro eroticism Intelligence is sexy Electro...,Metal
3,Inhaleexhale Inhale the exhaled Inhaleexhale I...,Metal
4,Under the dark sky we stand Under heavy rain w...,Metal
...,...,...
816003,[Verse 1]\nLooks like we made it\nLook how far...,country
816004,'Neath the spreading magnolia\nIn the Tennesse...,country
816005,Many years have passed it seems\nAnd now I'm a...,country
816007,[Verse 1]\nLonely is a rainy day\nWhen home is...,country


In [134]:
print(lyrics_df.groupby('genre').size())
lyrics_df = lyrics_df.sample(frac=1, random_state=12).sort_values(by='genre')

metal_df_train = lyrics_df[lyrics_df['genre']=='Metal'][:100000]
rock_df_train = lyrics_df[lyrics_df['genre']=='rock'][:100000]
rap_df_train = lyrics_df[lyrics_df['genre']=='rap'][:100000]
pop_df_train = lyrics_df[lyrics_df['genre']=='pop'][:100000]
country_df_train = lyrics_df[lyrics_df['genre']=='country'][:100000]

lyrics_train_df = pd.concat([metal_df_train, rock_df_train, rap_df_train, pop_df_train, country_df_train], ignore_index = True)
print(lyrics_train_df.groupby('genre').size())

metal_df_test = lyrics_df[lyrics_df['genre']=='Metal'][-10000:]
rock_df_test = lyrics_df[lyrics_df['genre']=='rock'][-10000:]
rap_df_test = lyrics_df[lyrics_df['genre']=='rap'][-10000:]
pop_df_test = lyrics_df[lyrics_df['genre']=='pop'][-10000:]
country_df_test = lyrics_df[lyrics_df['genre']=='country'][-10000:]

lyrics_test_df = pd.concat([metal_df_test, rock_df_test, rap_df_test, pop_df_test, country_df_test], ignore_index = True)
print(lyrics_test_df.groupby('genre').size())


genre
Metal      172565
country    133753
pop        121817
rap        123435
rock       121734
dtype: int64
genre
Metal      100000
country    100000
pop        100000
rap        100000
rock       100000
dtype: int64
genre
Metal      10000
country    10000
pop        10000
rap        10000
rock       10000
dtype: int64


In [136]:
lyrics_train_df.to_csv('./data/lyrics_train.csv')
lyrics_test_df.to_csv('./data/lyrics_test.csv')